In [65]:
import mysql.connector

# RDS instance details
db_endpoint = 'database2.cnunqj562ud9.us-east-1.rds.amazonaws.com'
db_port = 3306
db_user = 'admin'
db_password = 'admin123'
db_name = 'linkedinjobposting_lab1'


def print_query_data(query):
    cursor.execute(query)
    # Fetch all the results
    results = cursor.fetchall()
    # Process the results
    for row in results:
        print(row)

def add_triggers(query):
    cursor.execute(query)

# Define the name of the stored procedure you want to create
procedure_name = "GetSkillsAbrByCompany"
# Define the SQL statement for the stored procedure
procedure_sql = """
CREATE PROCEDURE {0}(IN companyID_input BIGINT)
BEGIN
   SELECT name AS company_name, skill_abr, COUNT(js.job_id) AS job_count
   FROM JobSkills js
   JOIN JobPosting jp ON js.job_id = jp.job_id
   JOIN Companies c ON jp.company_id = c.company_id
   WHERE c.company_id = companyID_input
   GROUP BY skill_abr, company_name
   ORDER BY job_count DESC;
END;
""".format(procedure_name)

# Define the name of the stored procedure you want to create
procedure_name1 = "UpdateJobPostingSalary1"
# Define the SQL statement for the stored procedure
procedure_sql1 = """
CREATE PROCEDURE {0}(IN p_job_ID BIGINT, newSalary DECIMAL(10,2))
BEGIN 
	UPDATE JobPosting
    SET max_salary = newSalary 
    WHERE job_id = p_job_ID;
END
""".format(procedure_name1)

# This trigger ensures that if someone tries to update a job posting with an expiry date that is earlier than the current expiry date, the closed_time is updated to the current time, indicating that the job posting is now closed.

CLOSED_JOB_POSTING_TRIGGER= """
    CREATE TRIGGER BeforeJobPostingUpdate
    BEFORE UPDATE ON JobPosting
    FOR EACH ROW
    BEGIN
        IF NEW.expiry < OLD.expiry THEN
            SET NEW.closed_time = NOW();
        END IF;
    END;
"""

# This trigger creates a log entry in the JobPostingLog table whenever a new job posting is inserted into the Jobposting table. The log entry includes the job_id, company_id, title, and the timestamp of the insertion.

JOB_POSTING_LOG_TRIGGER = """
CREATE TRIGGER AfterJobPostingInsert
AFTER INSERT ON JobPosting
FOR EACH ROW
BEGIN
  INSERT INTO JobPostingLog (job_id, company_id, title, inserted_at)
  VALUES (NEW.job_id, NEW.company_id, NEW.title, NOW());
END;
"""


# In summary, this trigger creates a notification in the NotificationLog table whenever skills associated with a job are updated in the Jobskills table. The notification includes a message indicating the job ID and specifies the recipient as the HR Department. The timestamp of the notification is also recorded.

NOTIFICATION_LOG_TRIGGER = """
CREATE TRIGGER AfterSkillsUpdate
AFTER UPDATE ON JobSkills
FOR EACH ROW
BEGIN
  INSERT INTO NotificationLog (message, recipient, sent_at)
  VALUES ('Skills for job ' or NEW.job_id or ' have been updated.', 'HR Department', NOW());
END;
"""

# This trigger creates a notification in the NotificationLog table whenever a new skill is inserted into the Jobskills table. The trigger checks if the skill abbreviation is 'RemoteWork'. If it is, a notification is created, indicating that remote work skills have been added for a specific job. The recipient is set to the 'Remote Work Department', and the timestamp of the notification is also recorded.

NEW_SKILL_INSERT_TRIGGER = """
CREATE TRIGGER AfterSkillsForRemoteWorkingInsert
AFTER INSERT ON JobSkills
FOR EACH ROW
BEGIN
  IF NEW.skill_abr = 'RemoteWork' THEN
    INSERT INTO NotificationLog (message, recipient, sent_at)
    VALUES ('Remote work skills added for job ' OR NEW.job_id, 'Remote Work Department', NOW());
  END IF;
END;
"""


MAXMIN_SALARY = """
        WITH ITCompanies AS (
            SELECT c.state, jp.max_salary, jp.min_salary
            FROM Companies c
            JOIN CompanyIndustries ci ON c.company_id = ci.company_id
            JOIN JobPosting jp ON c.company_id = jp.company_id
            WHERE ci.industry = 'Nonprofit Organization Management'
        )
        SELECT state,
            max_salary AS max_salary,
            min_salary AS min_salary,
            AVG(max_salary) OVER (PARTITION BY state) AS avg_max_salary,
            AVG(min_salary) OVER (PARTITION BY state) AS avg_min_salary
        FROM ITCompanies;
        """

COMPANIES_SPECIFIC_CITY = """
        WITH CompanyEmployeeCounts AS (
            SELECT c.name, c.city, ec.employee_count,
           AVG(ec.employee_count) OVER (PARTITION BY c.city) AS avg_employee_count
            FROM Companies c
            LEFT JOIN EmployeesCount ec ON c.company_id = ec.company_id
        )   
        SELECT name, city, avg_employee_count
            FROM CompanyEmployeeCounts
            WHERE city = 'New York';
        """

JOBROLL_SPECIFIC_SPECIALITY = """
        WITH CompanyJobSkills AS (
        SELECT c.name AS 'Job Roll', cs.speciality, js.skill_abr,
            ROW_NUMBER() OVER (PARTITION BY c.name, cs.speciality, js.skill_abr) AS rn
            FROM Companies c
            JOIN CompanySpecialities cs ON c.company_id = cs.company_id
            JOIN JobPosting jp ON c.company_id = jp.company_id
            JOIN JobSkills js ON jp.job_id = js.job_id
        )
        SELECT 'Job Roll', speciality, skill_abr
            FROM CompanyJobSkills
            WHERE speciality = 'Financial Services' AND skill_abr = 'ACCT'
            AND rn = 1
            ORDER BY 'Job Roll';
        """

FIRST_CTE_BENEFITS = """
        WITH num_benefits as (
	    SELECT job_id, count(distinct type) as num_benefits 
	        FROM Benefits 
	        GROUP BY 1
        ), 
        -- # Getting avg number of benefits in the next CTE
        avg_benefits as (
        SELECT avg(num_benefits) as avg_benefits_count 
            FROM num_benefits
        )
        SELECT n.job_id, jp.company_id, c.name as company_name, jp.title, num_benefits 
            FROM num_benefits n 
            INNER JOIN JobPosting jp 
            ON n.job_id = jp.job_id 
            INNER JOIN Companies c
            ON  jp.company_id = c.company_id
            WHERE num_benefits >= (select avg_benefits_count from avg_benefits)
            ORDER BY num_benefits desc;
        """


AVERAGE_SALARY_BY_EACH_INDUSTRY = """
        with avg_salary_by_industry as (
        select industry,  round(avg(min_salary),3) as avg_salary 
            from JobPosting jp  
            inner join CompanyIndustries ci 
            on jp.company_id = ci.company_id 
            where industry is not null 
            and coalesce(min_salary,0) > 0
            group by 1
        ), 
        ranking_salary as (
        select industry, avg_salary, 
            dense_rank() over (order by avg_salary desc) as salary_rank
            from avg_salary_by_industry
        ) 
        select * from ranking_salary where salary_rank<=3
            order by salary_rank;
        """

TOP_SKILL_BY_EACH_INDUSTRY = """
        with industry_skill_count as (
	    select ci.industry as company_industry, jp.job_id, js.skill_abr
	        from JobSkills js
	        inner join JobPosting jp 
	        on js.job_id = jp.job_id 
	        inner join CompanyIndustries ci
	        on jp.company_id = ci.company_id
	        group by 1,2,3
        ), 
        skill_count_raw as (
        select company_industry, skill_abr, 
            count(distinct job_id) as job_count 
            from industry_skill_count
            group by 1,2
        ), 
        skill_ranking as (
        select company_industry, skill_abr, job_count,
            dense_rank() over (partition by company_industry order by job_count desc) as skill_rank
            from skill_count_raw
        ) 
        select * from skill_ranking where skill_rank=1
            order by company_industry, job_count desc;
        """

MAX_EMPLOYEE_GROWTH_COUNT = """ 
        with max_min_emp_count_raw as (
        select company_id, 
        # Casting varchar to timestamp in case data type is not already datetime
            cast(time_recorded_ts as datetime) as date_time_stamp, 
            employee_count 
            from EmployeesCount
            where time_recorded_ts is not null 
            and coalesce(employee_count,0)>0
            group by 1,2,3
        ), 
        min_max_rank as (
            select company_id, date_time_stamp, employee_count, 
        # Using row number since we want unique row for a rank, dense rank may show duplicates
        # Creating 2 ranks - 1 for earliest and 1 for latest
            row_number() over (partition by company_id order by date_time_stamp) as min_rank, 
            row_number() over (partition by company_id order by date_time_stamp desc) as max_rank
            from  max_min_emp_count_raw
        ),
        
        growth_calc as (
            select coalesce(mn.company_id, mx.company_id) as company_id, 
        # Including condition to make growth % as 0 if denominator is 0, else do the actual percentage change
            case when coalesce(mn.earliest_emp_count,0)>0 then 
	                (coalesce(mx.latest_emp_count,0) - coalesce(mn.earliest_emp_count,0))*100/coalesce(mn.earliest_emp_count,0)
                     else 0 end as emp_count_growth_change_percentage from
                    (select company_id, employee_count as earliest_emp_count from min_max_rank where min_rank=1) mn 
                    left join 
                    (select company_id, employee_count as latest_emp_count from min_max_rank where max_rank=1) mx 
                    on mn.company_id = mx.company_id
        )
            select c.name as company_name, gc.* from 
            growth_calc gc inner join Companies c 
            on gc.company_id = c.company_id 
            order by emp_count_growth_change_percentage desc;
            """

JOB_OPENINGS_WITH_HIGHEST_NUMBER_OF_OPENINGS ="""
            select * from
            (
	        select ci.industry as company_industry, 
                count(distinct job_id) as jobs_count
	            from JobPosting jp 
	            inner join CompanyIndustries ci
	            on jp.company_id = ci.company_id
                where jp.closed_time is not null
	            group by 1
                order by jobs_count desc
            ) s
            limit 1;
            """

FULL_TIME_JOBS = """
            with required_benefits_jobs as (
	        select job_id 
	            from Benefits 
	            where lower(trim(type)) in ('401k', 'medical insurance', 'dental insurance')
	            group by 1
            )
            select count(distinct jp.job_id) as jobs_count 
                from JobPosting jp inner join Companies c
                on jp.company_id = c.company_id 
                inner join required_benefits_jobs ben 
                on jp.job_id = ben.job_id
                where lower(trim(c.state)) in ('ca', 'california'); """

HIGHEST_NUMBER_OF_SPECIALITIES ="""
            with spec_count as (
	        select industry, c.company_id, c.name as company_name,
	            count(distinct speciality) as specialities_count 
	            from CompanySpecialities cs inner join Companies c
	            on cs.company_id = c.company_id
                inner join CompanyIndustries ci 
                on ci.company_id = c.company_id
	            group by 1,2,3
            ),
            spec_rank as (
	        select sc.*, 
	            dense_rank() over (partition by industry order by specialities_count desc) as sp_rank
	            from spec_count sc
            )
            select * from spec_rank 
                where sp_rank=1 
                order by specialities_count desc;
                """

COUNT_OF_INTERNSHIPS = """
        select c.name as company_name, c.company_size, jp.description as job_description,
            count(distinct job_id) as jobs_count
            from JobPosting jp 
            inner join Companies c
            on jp.company_id = c.company_id
            where jp.closed_time is not null
            and lower(trim(jp.work_type)) = 'internship'
            group by 1,2,3 
            order by jobs_count desc;
            """


try:
    # Connect to the RDS database
    connection = mysql.connector.connect(
        host=db_endpoint,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name
    )

    if connection.is_connected():
        print(f"Connected to {db_endpoint} on port {db_port} as user {db_user} to database {db_name}")
        
        # Create a cursor for database operations
        cursor = connection.cursor()

        # Execute Procedure
        print("create stored procedure for skills by company and call the procedure")
        add_triggers(procedure_sql)
        cursor.callproc(procedure_name,[1016])
        print("create stored procedure for Updating new salary to max salary and call the procedure")
        add_triggers(procedure_sql1)
        cursor.callproc(procedure_name1,[133114754, 80000.00])
        # Execute Triggers
        print("This trigger creates a notification in the NotificationLog table whenever skills associated with a job are updated in the Jobskills table.")
        add_triggers(NOTIFICATION_LOG_TRIGGER)
        print("This trigger creates a notification in the NotificationLog table whenever a new skill is inserted into the Jobskills table.")
        add_triggers(NEW_SKILL_INSERT_TRIGGER)
        print("This trigger creates a log entry in the JobPostingLog table whenever a new job posting is inserted into the Jobposting table.")
        add_triggers(JOB_POSTING_LOG_TRIGGER)
        print("This trigger indicates that the job posting is now closed.")
        add_triggers(CLOSED_JOB_POSTING_TRIGGER)
        # Define your SQL SELECT statement
        print ("Calculate average maximum and minimum salaries for Nonprofit Organization Management companies while preserving the details of each company within the state")
        print_query_data(MAXMIN_SALARY)
        print ("List all companies in a specific city and their average employee counts.")
        print_query_data(COMPANIES_SPECIFIC_CITY)
        print ("Find companies job roll with a specific speciality that also have job postings with a certain skill requirement:")
        print_query_data(JOBROLL_SPECIFIC_SPECIALITY)
        print("Average # benefits per job and details of jobs offering higher than the average # of benefits")
        print_query_data(FIRST_CTE_BENEFITS)
        print ("Find the average salary by each job industry. Rank to find the top 3 industries by avg salary (joins and window function)")
        print_query_data(AVERAGE_SALARY_BY_EACH_INDUSTRY)
        print("Find the top skill for each job industry")
        print_query_data(TOP_SKILL_BY_EACH_INDUSTRY)
        print ("Which company has shown max employee count growth % over time? (headcount at latest ts - head count at earliest ts)/head count at earliest ts deduping and excluding null values")
        print_query_data(MAX_EMPLOYEE_GROWTH_COUNT)
        print(" Which job industry has the highest number of openings, Highest number of openings would mean the job_id that don't have a closed time yet")
        print_query_data(JOB_OPENINGS_WITH_HIGHEST_NUMBER_OF_OPENINGS)
        print ("How many full time jobs offer medical, dental, and 401k as benefits anda have HQ in California? Filtering for all jobs with the requried benefits")
        print_query_data(FULL_TIME_JOBS)
        print ("Which companies have the highest number of specialties by industry?")
        print_query_data(HIGHEST_NUMBER_OF_SPECIALITIES)
        print("What is the count of internships available with company name, size and job description?")
        print_query_data(COUNT_OF_INTERNSHIPS)
        
        
        
        
        

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # Close the cursor and connection
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()
        print("Database connection closed.")

    

Connected to database2.cnunqj562ud9.us-east-1.rds.amazonaws.com on port 3306 as user admin to database linkedinjobposting_lab1
create stored procedure for skills by company and call the procedure
create stored procedure for Updating new salary to max salary and call the procedure
This trigger creates a notification in the NotificationLog table whenever skills associated with a job are updated in the Jobskills table.
This trigger creates a notification in the NotificationLog table whenever a new skill is inserted into the Jobskills table.
This trigger creates a log entry in the JobPostingLog table whenever a new job posting is inserted into the Jobposting table.
This trigger indicates that the job posting is now closed.
Calculate average maximum and minimum salaries for Nonprofit Organization Management companies while preserving the details of each company within the state
('alabama', Decimal('37000'), Decimal('33000'), Decimal('37000.0000'), Decimal('33000.0000'))
('alabama', None, No